# imports

In [21]:
from azureml.core import Workspace


from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
import os

from azureml.core import Workspace,Environment, Experiment
from azureml.core.conda_dependencies import CondaDependencies


from azureml.core.runconfig import RunConfiguration


# Defining utility function

In [35]:
def getOrCreateCompute(ws:Workspace):
    
    from azureml.core.compute import AmlCompute
    from azureml.core.compute import ComputeTarget
    from azureml.core.compute_target import ComputeTargetException

    aml_compute_target = "testcot"
    try:
        aml_compute = AmlCompute(ws, aml_compute_target)
        print("found existing compute target.")
    except ComputeTargetException:
        print("creating new compute target")

        provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                    min_nodes = 0, 
                                                                    max_nodes = 4)    
        aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
        aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    finally:
        return aml_compute

In [34]:
def createRunConfig()-> RunConfiguration:
    
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    # create a new runconfig object
    run_config = RunConfiguration()
    mlclient= MLClient.from_config(DefaultAzureCredential())
    env = mlclient.environments.get(name="titanic-env", version="1")
    
    run_config.environment=env

    

    return run_config

# Build dev pipeline

In [24]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


# Default datastore (Azure blob storage)
def_blob_store = ws.get_default_datastore()
#def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))
#Upload file to datastore

# Use a CSV to read in the data set.
file_name = "../data/rawdata/train.csv"

if not os.path.exists(file_name):
    raise Exception(
        'Could not find CSV dataset at "%s". '
        % file_name
    )  # NOQA: E50
# Upload file to default datastore in workspace

mlops-aml-ws
mlops-rg
westeurope
f90533aa-280d-40b9-9949-a7ba0ee9511f
Blobstore's name: workspaceblobstore


# Uploading data to blob storage

In [25]:
target_path = "training-data/"
def_blob_store.upload_files(
    files=[file_name],
    target_path=target_path,
    overwrite=True,
    show_progress=False,
)

blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="test_data",
    path_on_datastore="training-data/train.csv")


aml_compute = getOrCreateCompute(ws)
run_config = createRunConfig()
#processed_data1 = PipelineData("processed_data1",datastore=def_blob_store)
train_data = PipelineData("train_data1",datastore=def_blob_store)

Found the config file in: /config.json


found existing compute target.


## Preprocess step

In [26]:
source_directory="../src/preprocess/"
preprocess_step = PythonScriptStep(
    script_name="preprocess.py", 
    arguments=["--data", blob_input_data],
    inputs=[blob_input_data],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

## Training step

In [27]:
source_directory="../src/train/"
train_step = PythonScriptStep(
    name="Train Model",
    script_name="train.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    outputs=[train_data],
    arguments=[
        "--model",
        train_data
    ],
    runconfig=run_config,
    allow_reuse=True,
)

## Validation step

In [28]:
source_directory="../src/evaluation/"
evaluate_step = PythonScriptStep(
    name="Evaluate Model ",
    script_name="eval.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    inputs=[train_data],
    arguments=[
        "--model_path",
        train_data,
    ],
    runconfig=run_config,
    allow_reuse=False,
)

# Pipeline creation

In [33]:
train_step.run_after(preprocess_step)
evaluate_step.run_after(train_step)
steps = [preprocess_step,train_step, evaluate_step]

train_pipeline = Pipeline(workspace=ws, steps=steps)
#train_pipeline._set_experiment_name


AttributeError: 'Environment' object has no attribute 'python'

In [ ]:
pipeline_run1 = Experiment(ws, 'titanic-pipeline').submit(train_pipeline)

In [ ]:
pipeline_run1.wait_for_completion(show_output=True)

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [ ]:
# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
  name=local_endpoint_name, description="this is a sample local endpoint"
)